In [1]:
DRAFT_MODEL="Qwen/Qwen2.5-Math-1.5B-Instruct"
TARGET_MODEL="Qwen/Qwen2.5-Math-7B-Instruct"
PRM="Skywork/Skywork-o1-Open-PRM-Qwen-2.5-1.5B"
DRAFT_IP_ADDRESS="http://localhost:12340/v1"
TARGET_IP_ADDRESS="http://localhost:12341/v1"
PRM_IP_ADDRESS="http://localhost:12342/v1"

In [2]:
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import numpy as np

openai_api_key = "EMPTY"

args = {
    "draft_model_path_rsd": DRAFT_MODEL,
    "draft_model_ip_address": DRAFT_IP_ADDRESS,
    "target_model_path_rsd": TARGET_MODEL,
    "target_model_ip_address": TARGET_IP_ADDRESS,
    "prm_ip_address_rsd": PRM_IP_ADDRESS,
    "prm_path_rsd": PRM
}


/data/mert_cemri/miniconda3/envs/spec/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# draft_client = OpenAI(
#         api_key=openai_api_key,
#         base_url=args["draft_model_ip_address"],
#     )
# draft_tokenizer = AutoTokenizer.from_pretrained(args["draft_model_path_rsd"], trust_remote_code=True)

target_client = OpenAI(
    api_key=openai_api_key,
    base_url=args["target_model_ip_address"],
)
target_tokenizer = AutoTokenizer.from_pretrained(args["target_model_path_rsd"], trust_remote_code=True)

prm_client = OpenAI(
    api_key=openai_api_key,
    base_url=args["prm_ip_address_rsd"],
)
prm_tokenizer = AutoTokenizer.from_pretrained(args["prm_path_rsd"], trust_remote_code=True)

## Check if we can get the logits from the target model

In [4]:
input_text = "Hello, how are you?"
input_ids = target_tokenizer.encode(input_text)

prm_model_logits = prm_client.embeddings.create(
                model=args["prm_path_rsd"].split("/")[-1],
                input=input_ids,
            )

In [5]:
prm_model_logits.data[0].embedding

[-0.283203125, -0.271484375, -1.0, -1.296875, -2.328125, 0.291015625]

In [ ]:
# input_text = "Hello, how are you?"
# input_ids = target_tokenizer.encode(input_text)

# target_model_logits = target_client.embeddings.create(
#                 model=args["target_model_path_rsd"].split("/")[-1],
#                 input=input_ids,
#             )

In [6]:
from vllm import LLM

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# Load the target model
target_model = LLM(
    model="Qwen/Qwen2.5-Math-7B-Instruct",
    tensor_parallel_size=1,
    task="embed"
)


2025-02-28 01:50:09,871	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 02-28 01:50:10 __init__.py:207] Automatically detected platform cuda.
INFO 02-28 01:50:10 __init__.py:30] Available plugins for group vllm.general_plugins:
INFO 02-28 01:50:10 __init__.py:32] name=register_dummy_model, value=vllm_add_dummy_model.prm_model:register
INFO 02-28 01:50:10 __init__.py:34] all available plugins for group vllm.general_plugins will be loaded.
INFO 02-28 01:50:10 __init__.py:36] set environment variable VLLM_PLUGINS to control which plugins to load.
ERROR 02-28 01:50:10 __init__.py:46] Failed to load plugin register_dummy_model
ERROR 02-28 01:50:10 __init__.py:46] Traceback (most recent call last):
ERROR 02-28 01:50:10 __init__.py:46]   File "/data/mert_cemri/miniconda3/envs/spec/lib/python3.11/site-packages/vllm/plugins/__init__.py", line 42, in load_plugins_by_group
ERROR 02-28 01:50:10 __init__.py:46]     func = plugin.load()
ERROR 02-28 01:50:10 __init__.py:46]            ^^^^^^^^^^^^^
ERROR 02-28 01:50:10 __init__.py:46]   File "/data/mert_cemri/minico

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.86it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.73it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.02it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.88it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.88it/s]



INFO 02-28 01:50:20 model_runner.py:1115] Loading model weights took 14.2419 GB


In [7]:
encodings = target_model.embed("What is the capital of France?")

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.99it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


In [11]:
encodings

[EmbeddingRequestOutput(request_id='0', outputs=EmbeddingOutput(hidden_size=3584), prompt_token_ids=[3838, 374, 279, 6722, 315, 9625, 30], finished=True)]

In [14]:
encodings[0].outputs

EmbeddingOutput(hidden_size=3584)

: 

## Other checks

In [4]:
config = {
    "draft_model_name_or_path": DRAFT_MODEL,
    "temperature": 1.0,
    "top_p": 0.9,
    "max_tokens": 2048
}

In [5]:
gen_prompts = [
            "Say hello",
            "Say goodbye"
        ]
llm = draft_client

In [6]:
draft_responses = llm.completions.create(
                model=config["draft_model_name_or_path"].split("/")[-1],
                prompt=gen_prompts,
                temperature=config["temperature"],
                top_p=config["top_p"],
                max_tokens=config["max_tokens"],
                stop=["\n\n"],
                n=1,
                stream=False,
                extra_body={
                    "include_stop_str_in_output": True
                                   }
            ).choices
llm_outputs = sorted(draft_responses, key=lambda x: int(x.index))

In [ ]:
print("llm_outputs: \n", llm_outputs)

In [ ]:
for output in llm_outputs:
    print(f"output.text: {output.text}")
    print(f"output.index: {output.index}")
    print(f"output.finish_reason: {output.finish_reason}")
    print(f"output.logprobs: {output.logprobs}")

In [9]:
def prepare_input(problem, response, tokenizer, step_token):
    prompt_ids = tokenizer.encode(tokenizer.bos_token + problem + "\n")
    response_ids = []
    steps = []
    reward_flags = [0] * len(prompt_ids)
    step_token_id = tokenizer.encode(step_token)[-1]
    for idx, step in enumerate(response.split(step_token)):
        if step != "":
            step_ids = tokenizer.encode(step)
        else:
            step_ids = []
        step_ids += [step_token_id]
        step = step + step_token
        flag = [0] * len(step_ids)
        flag[-1] = 1
        response_ids.extend(step_ids)
        reward_flags.extend(flag)
        steps.append(step)
    input_ids = prompt_ids + response_ids
    return input_ids, steps, reward_flags

In [11]:
processed_data = [
    prepare_input(p, full_resp.text, tokenizer=prm_tokenizer, step_token="\n\n") 
    for p, full_resp in zip(gen_prompts, llm_outputs)
]
input_ids, steps, reward_flags = zip(*processed_data)

In [31]:
rewards = prm_client.embeddings.create(
                model=args["prm_path_rsd"].split("/")[-1],
                input=input_ids,
            )

In [ ]:
for i in rewards:
    print(i)

In [35]:
def sigmoid(x):
    return 1/(np.exp(-x) + 1)
    
def derive_step_rewards_vllm(raw_rewards, batch_reward_flags):
    batch_step_rewards = []
    for idx,data in enumerate(raw_rewards.data):
        rewards = data.embedding
        reward_flags = batch_reward_flags[idx]

        step_rewards = [sigmoid(reward) for reward,flag in zip(rewards,reward_flags) if flag == 1]   
        batch_step_rewards.append(step_rewards)
    return batch_step_rewards

In [ ]:
step_rewards = derive_step_rewards_vllm(rewards, reward_flags)
step_rewards

In [ ]:
merged_model_path = "/data/user_data/mert/spec/models_merged/Qwen2--qwen_7b_merged"
reward_model_name = "Qwen/Qwen2.5-Math-PRM-7B"
causal_model_name = "Qwen/Qwen2.5-7B-Instruct"